In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import glob
%cd ../

/mnt/c/Users/Daniel Yang/Desktop/Final_Project


In [2]:
fileId_path = "experiments/fileIds.pkl"
queryInfo_path = "experiments/queryInfo.pkl"
timeData_path = "experiments/timeData.pkl"
audioFiles_path = "experiments/audioFiles.pkl"
with open(fileId_path, 'rb') as f:
    fileIds=pkl.load(f)
with open(queryInfo_path, 'rb') as f:
    queryInfo=pkl.load(f)
with open(timeData_path, 'rb') as f:
    timeData=pkl.load(f)
with open(audioFiles_path, 'rb') as f:
    audioFiles=pkl.load(f)

In [3]:
def readHypFiles(hypdir):
    data = []
    for hypfile in sorted(glob.glob("experiments/{}/*.hyp".format(hypdir))):
        print(hypfile)
        with open(hypfile, "rb") as f:
            data.append(pkl.load(f))
    return data

In [4]:
def evaluateSegmentLevel(segment_predict, solo_name, ref_name, sr=22050):
    pieceName = solo_name.split('_')[0]
    gt_segments = queryInfo[pieceName]
    gt_samples = []
    diffs = []
    for beats in gt_segments:
        beat1 = beats.split(',')[0][1:]
        beat2 = beats.split(',')[1][:-1]
        sample1 = timeData[ref_name][beat1]*512
        sample2 = timeData[ref_name][beat2]*512
        gt_samples.append((sample1, sample2))
    for idx,segment in enumerate(segment_predict):
        seg0 = segment[0]
        seg1 = segment[1]
        diff0 = np.abs(gt_samples[idx][0]-seg0)/sr
        diff1 = np.abs(gt_samples[idx][1]-seg1)/sr
        diffs.append(diff0)
        diffs.append(diff1)
    return diffs

In [63]:
def evaluateBeatLevel(tf, segment_predict, solo_name, ref_name, sr=22050):
    # Only evaluate within segments
    diffs = []
    pieceName = solo_name.split('_')[0]
    gt_segments = queryInfo[pieceName]
    annotated_beats = list(timeData[ref_name].keys())
    for beats in gt_segments:
        beat1 = beats.split(',')[0][1:]
        beat2 = beats.split(',')[1][:-1]
        idx1 = annotated_beats.index(beat1)
        idx2 = annotated_beats.index(beat2)
        base_time = timeData[ref_name][beat1]
        new_times = {}
        for i in range(idx1,idx2):
            key = annotated_beats[i]
            sample = (timeData[ref_name][key]-base_time)*sr
            new_time = tf[int(sample)]
            new_times[key]=new_time
            
        gt_beats = list(timeData[solo_name].keys())
        gt_base_time=timeData[solo_name][beat1]
        for key in new_times:
            if key in gt_beats:
                gt_time = (timeData[solo_name][key]-gt_base_time)*sr
                pred_time = new_times[key]
                #print(gt_time, pred_time)
                diffs.append(np.abs(gt_time-pred_time))
    return diffs

In [68]:
def evaluate(hypdir, sr=22050):
    data = readHypFiles(hypdir)
    times = []
    segment_tolerances =[1]
    segment_total = np.zeros(len(segment_tolerances))
    segment_correct = np.zeros(len(segment_tolerances))
    
    beat_tolerances =[.25]
    beat_total = np.zeros(len(beat_tolerances))
    beat_correct = np.zeros(len(beat_tolerances))
    for solo_name, ref_name, tf, segment_predict, time_taken in data:
        segment_predict = np.array(segment_predict)*512
        segment_diffs = evaluateSegmentLevel(segment_predict, solo_name, ref_name)
        beat_diffs = evaluateBeatLevel(tf, segment_predict, solo_name, ref_name)
        segment_diffs =np.array(segment_diffs)/sr
        beat_diffs =np.array(beat_diffs)/sr
        times.append(time_taken)
        
        #Update Segment Level Accuracy
        for idx, tolerance in enumerate(segment_tolerances):
            segment_correct[idx] += np.sum(np.where(segment_diffs<tolerance, 1, 0))
            segment_total[idx]+=len(segment_diffs)
            
            
        #Update Beat Level Accuracy
        for idx, tolerance in enumerate(beat_tolerances):
            beat_correct[idx] += np.sum(np.where(beat_diffs<tolerance, 1, 0))
            beat_total[idx]+=len(beat_diffs)
    print(segment_correct/segment_total)
    print(beat_correct/beat_total)
        

In [70]:
evaluate("debug")

experiments/debug/p1s_perf25-p1f_perf16.hyp
experiments/debug/p1s_perf25-p1f_perf23.hyp
0.0 1
22248.33333344998 18978
41105.33333864998 37190
59750.66666939999 54562
80238.00000089999 71542
99848.33333159999 87407
120988.99999725001 104338
142035.99999749998 121964
161534.0000001 139823
181809.66667185 157283
202401.33332684997 174851
221472.00000030003 191504
241578.9999966 208435
262288.33333815 224901
284107.6666638 241820
305355.66666570003 259564
325481.6666619 277449
346601.6666637 295474
367222.6666689 311620
387946.3333263001 326811
410126.33332755003 352563
430343.6666655 374886
450706.99999920005 395802
476698.33332765 418561
497971.33332555 437044
517971.9999971999 455244
538714.6666692 481033
558521.6666620501 498394
578813.6666655 516455
601029.6666654 537392
0.0 1
20452.66665959991 19070
40647.000002700035 37255
58471.66666529982 55665
78101.66666294984 73295
98056.00000035 89381
119029.66666185002 106749
139125.33332564982 124825
158793.00000059998 142961
178683.99999764